# Generate demand timeseries per geo

- This notebook feches creates a demand/load profile for a given year (target year).
- It uses a normalized demand profile and parameters dictating total annual energy to do this.
- The output data is used by generator if it exists and otherwise interpolated data is used. (TODO: code this functionality)
- **NOTE: This notebook uses functions in order to facilitate moving these functions into generator when useful.**

In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import json
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent.parent
sys.path.append(str(root_path))

import paths

logging.basicConfig(level=logging.INFO)

In [2]:
# Load the generator config that we base the calculations on, or else create this config in notebook

config_path = paths.generator_path / 'configs' / 'default.json'

if config_path.is_file():
    with open(config_path, "r") as f:
        config = json.load(f)
else:
    config = {
        'base-year': 2024,
        'demand': {
            'base-demand': 19000000,
            'projection': {
                '2030': 34000000,
                '2040': 49000000
            },
            'growth-only': True
        },
        'scenarios': {
            'geography': ['14', '14:1488'],
            'target-year': [2030]
        }
    }

# Load the energy percentages per geo file and normalized load profile
energy_percentages = pd.read_csv(paths.demand_root / 'energy-percentages-vgr.csv.gz', compression='gzip', index_col='Kod')
normalized_demand = pd.read_csv(paths.demand_root / 'normalized-demand-2023-3h.csv', index_col='timestamp')

In [3]:
# A function that either extrapolates the demand for target years beyond the projection, or else interpolates the demand for target years inside the projection

def interpolate_demand(target_year, base_year, base_demand, projection):
    # Build a dataframe out of the config energy parameters
    energy_t = pd.DataFrame([base_demand] + list(projection.values()), index=([base_year] + [int(key) for key in projection.keys()]), columns=['energy'])
    energy_t = energy_t.sort_index()

    # If target year falls beyond the last year in the projection, extrapolate from the final two values
    if target_year > energy_t.index.max():
        return energy_t.iloc[-1]['energy'] + (target_year - energy_t.index[-1]) * (energy_t.iloc[-1]['energy'] - energy_t.iloc[-2]['energy']) / (energy_t.index[-1] - energy_t.index[-2])

    # Else, use pandas interpolation to calculate the value
    # Add the target_year to the dataframe with an empty 'energy' value if the year is not already in the projection
    if target_year not in energy_t.index: energy_t.loc[target_year] = None
    energy_t = energy_t.sort_index()

    # Interpolate missing values
    energy_t['energy'] = energy_t['energy'].interpolate(method='linear')

    return energy_t.loc[target_year, 'energy']

In [4]:
# A function that calculates the demand time series

def project_annual_demand(target_year, base_year, base_demand, projection, growth_only, percentage):
    return (interpolate_demand(target_year, base_year, base_demand, projection) - (1 if growth_only else 0) * base_demand) * percentage

In [5]:
# Iterate through the configuration and output projected-demand for each geography and target_year combo

base_year = config['base-year']
base_demand = config['demand']['base-demand']
demand_projection = config['demand']['projection']
growth_only = config['demand']['growth-only']
geography = config['scenarios']['geography']
target_year = config['scenarios']['target-year']

# Iterate through geography and target_year and write each demand profile to file
for geo in geography:
    for year in target_year:
        energy_percentage = energy_percentages.loc[geo, 'percentage']
        projected_demand = project_annual_demand(year, base_year, base_demand, demand_projection, growth_only, energy_percentage) * normalized_demand
        projected_demand.to_csv(paths.demand / f"projected-demand,geography={geo},target-year={year},growth-only={growth_only}.csv.gz", compression='gzip')